Import Libraries

In [1]:
import pandas as pd
import numpy as np

Load the dataset

In [2]:
data = pd.read_csv("Kangaroo_main.csv")

Drop irrelevant columns for price prediction

In [3]:
drop_cols = data.drop(columns=[
    "url",
    "id",
    "hasBalcony",
    "floorCount",
    "diningRoomSurface",
    "monthlyCost",
    "hasGarden",
    "hasTerrace",
    "accessibleDisabledPeople",
    "Unnamed: 0",
    "terraceOrientation",
    "hasDressingRoom",
    "hasDiningRoom",
    "streetFacadeWidth",
    "hasHeatPump",
    "hasPhotovoltaicPanels",
    "hasThermicPanels",
    "hasLivingRoom",
    "gardenOrientation",
    "parkingCountOutdoor",
    "hasArmoredDoor",
    "hasVisiophone"
    ])

Drop Big Values (Outliers) and fillna some of them with the median values 

In [4]:
drop_out = drop_cols
drop_out.loc[drop_out["bedroomCount"] >= 100, "bedroomCount"] = np.nan
median_bedrooms = drop_out["bedroomCount"].median()
drop_out["bedroomCount"].fillna(median_bedrooms, inplace=True)
# ..............................................................
drop_out.loc[drop_out["bathroomCount"] >= 100, "bathroomCount"] = np.nan
median_bathrooms = drop_out["bathroomCount"].median()
drop_out["bathroomCount"].fillna(median_bathrooms, inplace=True)
#...............................................................
drop_out.loc[drop_out["habitableSurface"] >= 600,"habitableSurface"] = np.nan
median_hS = drop_out["habitableSurface"].median()
drop_out["habitableSurface"].fillna(median_hS, inplace=True)
#...............................................................
drop_out.loc[drop_out["toiletCount"] >= 25, "toiletCount"] = 1
drop_out.loc[drop_out["landSurface"] >= 1000, "landSurface"] = 0
drop_out.loc[drop_out["gardenSurface"] >= 500, "gardenSurface"] = 0
drop_out.loc[drop_out["terraceSurface"] >= 250, "terraceSurface"] = 0
drop_out.loc[drop_out["parkingCountIndoor"] >= 10, "parkingCountIndoor"] = 1

C:\Users\Administrateur\AppData\Local\Temp\ipykernel_21024\3379475542.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  drop_out["bedroomCount"].fillna(median_bedrooms, inplace=True)
C:\Users\Administrateur\AppData\Local\Temp\ipykernel_21024\3379475542.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values al

Fill the missing values in these columns ("gardenSurface, terraceSurface) with 0 
    And fill the missing values in (habitableSurface, bathroomCount and bedroomCount) with their median

In [5]:
drop_cols["gardenSurface"] = drop_cols["gardenSurface"].fillna(0)    # Replace NaNs with 0
drop_cols["terraceSurface"] = drop_cols["terraceSurface"].fillna(0)   # Replace NaNs with 0 # Replace NaNs with median value
drop_cols["bathroomCount"] = drop_cols["bathroomCount"].fillna(drop_cols["bathroomCount"].mode()) # Replace NaNs with median value
drop_cols["bedroomCount"] = drop_cols["bedroomCount"].fillna(drop_cols["bedroomCount"].mode()) # Replace NaNs with median value

Drop Nan Values in "price" and "habitableSurface" columns                                

In [6]:
drop_cols.dropna(subset=["price"], inplace=True)
drop_cols.dropna(subset=["habitableSurface"], inplace=True)

Merge type and subtype without making doubles

In [7]:
drop_cols["type_subtype"] = (drop_cols["type"].str.upper() + " " + drop_cols["subtype"].str.upper()) #concatenate type and subtype into a new column called type_subtype
drop_cols["type_subtype"] = drop_cols["type_subtype"].apply(lambda x: " " .join(sorted(set(x.split())))) # delete duplicates in the type_subtype column
drop_cols.drop(columns=["type"], inplace=True)
drop_cols.drop(columns=["subtype"], inplace=True)

Divide "type_subtype" in two columns "apartment_map_score" and "house_map_score" and map them

In [8]:
# Dictionary to assign a score to each apartment type based on its category
apartment_map_score = {
    'APARTMENT KOT': 0,
    'APARTMENT FLAT_STUDIO': 1,
    'APARTMENT GROUND_FLOOR': 2,
    'APARTMENT': 3,
    'APARTMENT DUPLEX': 4,
    'APARTMENT TRIPLEX': 5,
    'APARTMENT LOFT': 6,
    'APARTMENT SERVICE_FLAT': 7,
    'APARTMENT PENTHOUSE': 8
}
# Dictionary to assign a score to each house type based on its category
house_map_score = {
    'HOUSE': 0,
    'HOUSE MIXED_USE_BUILDING': 1,
    'COUNTRY_COTTAGE HOUSE': 2,
    'CHALET HOUSE': 3,
    'BUNGALOW HOUSE': 4,
    'HOUSE TOWN_HOUSE': 5,
    'FARMHOUSE HOUSE': 6,
    'HOUSE OTHER_PROPERTY': 7,
    'HOUSE VILLA': 8,
    'HOUSE PAVILION': 9,
    'HOUSE MANOR_HOUSE': 10,
    'EXCEPTIONAL_PROPERTY HOUSE': 11,
    'CASTLE HOUSE': 12
}
drop_cols['apartment_map_score'] = drop_cols['type_subtype'].map(apartment_map_score)
drop_cols['house_map_score'] = drop_cols['type_subtype'].map(house_map_score)

Deal with "locality" doubles and orthographic faults ...

In [9]:
from rapidfuzz import process, fuzz

# Convert both the 'locality' and 'province' columns to uppercase for uniform comparison
drop_cols["locality"] = drop_cols["locality"].str.upper()
drop_cols["province"] = drop_cols["province"].str.upper()

# Dictionary to store the matched pairs
standard_map = {}

# Function to find a match from the known standards
def find_match(name, standards, threshold=80):
    if not standards:
        return None  # No standards to compare to, return None

    # Use rapidfuzz to find the closest match to the 'name' from the 'standards' list using the token_sort_ratio scoring method
    result = process.extractOne(name, standards, scorer=fuzz.token_sort_ratio)
    
    if result is None:
        return None  # No match found, return None
    
    match, score, _ = result
    return match if score >= threshold else None  # Return match if score is greater than the threshold, else None

# List to store all standardized names encountered so far
standards = []

# Main loop to iterate through the 'locality' column
for name in drop_cols['locality']:
    match = find_match(name, standards)  # Try to find a match for the current locality
    if match:
        standard_map[name] = match  # If a match is found, store it in the dictionary
    else:
        standard_map[name] = name  # If no match is found, keep the name as it is
        standards.append(name)  # Add the name to the list of standards

# Create a new column with normalized locality names
drop_cols['locality_normalized'] = drop_cols['locality'].map(standard_map)

# Identify rows where the 'locality' has been changed to its normalized form
modifications = drop_cols[drop_cols['locality'] != drop_cols['locality_normalized']]

# Now, normalize the locality by concatenating the 'postCode' and 'locality' in uppercase
drop_cols["locality_normalized"] = (
    drop_cols["postCode"].astype(str).str.upper() + " - " + drop_cols["locality"].astype(str).str.upper()
)

# Drop the original 'locality' and 'postCode' columns as they are no longer needed
drop_cols.drop(columns=["locality"], inplace=True)
drop_cols.drop(columns=["postCode"], inplace=True)


Merge "postCode" with "province" and renaming the column from "postCode" to "postCode-province"

Fill "kitchenType" missing values with "INSTALLED" if there is a value in "kitchenSurface" and Drop rows with missing values in both columns

In [10]:
drop_cols.loc[drop_cols["kitchenSurface"].notna() & (drop_cols["kitchenSurface"] != "") & (drop_cols["kitchenType"].isna()), "kitchenType"] = "INSTALLED"
drop_cols = drop_cols[~(drop_cols["kitchenType"].isna() & drop_cols["kitchenSurface"].isna())]

Mapping of "floodZoneType" column

In [11]:
floodZoneMapping = {
    "NON_FLOOD_ZONE": 0,
    "POSSIBLE_N_CIRCUMSCRIBED_WATERSIDE_ZONE": 1,
    "CIRCUMSCRIBED_WATERSIDE_ZONE": 2,
    "POSSIBLE_N_CIRCUMSCRIBED_FLOOD_ZONE": 3,
    "POSSIBLE_FLOOD_ZONE": 4,
    "CIRCUMSCRIBED_FLOOD_ZONE": 5,
    "RECOGNIZED_FLOOD_ZONE": 6,
    "RECOGNIZED_N_CIRCUMSCRIBED_WATERSIDE_FLOOD_ZONE": 7,
    "RECOGNIZED_N_CIRCUMSCRIBED_FLOOD_ZONE": 8
}
drop_cols["floodZoneMapping"] = drop_cols["floodZoneType"].map(floodZoneMapping)

C:\Users\Administrateur\AppData\Local\Temp\ipykernel_21024\2513258033.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drop_cols["floodZoneMapping"] = drop_cols["floodZoneType"].map(floodZoneMapping)


Mapping of "epcScore" column

In [12]:
epcScoreMapping = {
    "A++": 1,
    "A+": 2,
    "A": 3,
    "B": 4,
    "C": 5,
    "D": 6,
    "E": 7,
    "F": 8,
    "G": 9
}
drop_cols["epcScoreMapping"] = drop_cols["epcScore"].map(epcScoreMapping)
drop_cols.fillna(0, inplace=True)  # Fill NaN values with 0 for epcScoreMapping and floodZoneMapping columns

C:\Users\Administrateur\AppData\Local\Temp\ipykernel_21024\2430905767.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drop_cols["epcScoreMapping"] = drop_cols["epcScore"].map(epcScoreMapping)
C:\Users\Administrateur\AppData\Local\Temp\ipykernel_21024\2430905767.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drop_cols.fillna(0, inplace=True)  # Fill NaN values with 0 for epcScoreMapping and floodZoneMapping columns


Mapping of "buildingCondition" column

In [13]:
buildingConditionMapping = {
    "AS_NEW": 1,
    "JUST_RENOVATED": 2,
    "GOOD": 3,
    "TO_RENOVATE": 4,
    "TO_BE_DONE_UP": 5,
    "TO_RESTORE": 6,
}
drop_cols["bulidingConditionMapping"] = drop_cols["buildingCondition"].map(buildingConditionMapping)
drop_cols["bulidingConditionMapping"].fillna(0, inplace=True)  

C:\Users\Administrateur\AppData\Local\Temp\ipykernel_21024\2790084325.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drop_cols["bulidingConditionMapping"] = drop_cols["buildingCondition"].map(buildingConditionMapping)
C:\Users\Administrateur\AppData\Local\Temp\ipykernel_21024\2790084325.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to 

Mapping of "heatingType" column

In [14]:
heatingTypeMapping = {
    "NOT_INSTALLED": 0,
    "SOLAR": 1,
    "WOOD": 2,
    "PELET": 3,
    "GAS": 4,
    "FUELOIL": 5,
    "ELECTRIC": 6,
    "CARBON": 7,
}
drop_cols["heatingTypeMapping"] = drop_cols["heatingType"].map(heatingTypeMapping)
drop_cols["heatingTypeMapping"].fillna(0, inplace=True)  

C:\Users\Administrateur\AppData\Local\Temp\ipykernel_21024\318446622.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drop_cols["heatingTypeMapping"] = drop_cols["heatingType"].map(heatingTypeMapping)
C:\Users\Administrateur\AppData\Local\Temp\ipykernel_21024\318446622.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operati

Drop epcScore (rangy) values (unprecise)

In [15]:
drop_epcScore_rows_with_two_letters = ['G_C', 'F_D', 'C_A', 'F_C', 'E_C', 'C_B', 'E_D', 'G_F', 'D_C', 'G_E', 'X']
drop_cols = drop_cols[~drop_cols['epcScore'].isin(drop_epcScore_rows_with_two_letters)]

Fill the epcScore NaN's by the median which is C(5)

In [16]:
drop_cols["epcScore"] = drop_cols["epcScore"].fillna(0)

Adding missing values to these (boolean) columns (that only have True values) as False values 

In [17]:
drop_cols["hasOffice"] = drop_cols["hasOffice"].fillna(False)
drop_cols["hasAirConditioning"] = drop_cols["hasAirConditioning"].fillna(False)
drop_cols["hasSwimmingPool"] = drop_cols["hasSwimmingPool"].fillna(False)
drop_cols["hasAttic"] = drop_cols["hasAttic"].fillna(False)
drop_cols["hasLift"] = drop_cols["hasLift"].fillna(False)
drop_cols["hasBasement"] = drop_cols["hasBasement"].fillna(False)
drop_cols["hasFireplace"] = drop_cols["hasFireplace"].fillna(False)


Remove columns with more than 4 facade and fill mean value that is (2)

In [18]:
drop_cols["facedeCount"] = drop_cols["facedeCount"].fillna(2) 
drop_cols = drop_cols[drop_cols['facedeCount'] < 5]

Delete columns with more 10 parkingCountIndoor and fill 0 for missing values

In [19]:
drop_cols["parkingCountIndoor"] = drop_cols["parkingCountIndoor"].fillna(0) 
drop_cols = drop_cols[drop_cols['parkingCountIndoor'] < 11]

Fillna missing "toiletCount" values with the median of their respective "type-subtype" group (appart/house/...)

In [20]:
drop_cols["toiletCount"] = drop_cols.groupby("type_subtype")["toiletCount"].transform(lambda x: x.fillna(x.median())) # Replace missing values with median value based on type-subtype group